# Transactional SQL

## 1. Introduction

### What is a Transaction?
A transaction in SQL is a sequence of one or more SQL statements that are executed as a single unit of work. Transactions ensure that the database remains in a consistent state, even in the presence of failures or errors. 

---

### Why Are Transactions Important?
Transactions are essential in database management systems because they:
- Maintain **data integrity**.
- Ensure **consistency** of the database.
- Allow **safe concurrent access** by multiple users.
- Enable recovery from unexpected failures.

---

### The ACID Properties
Transactions are guided by four key properties collectively known as **ACID**:

1. **Atomicity**  
   A transaction is all-or-nothing. If any part of the transaction fails, the entire transaction is rolled back, leaving the database unchanged.

2. **Consistency**  
   A transaction ensures that the database transitions from one valid state to another, maintaining the integrity of the data.

3. **Isolation**  
    Transactions running concurrently operate as if they are the only ones executing in the system, ensuring they do not interfere with each other.

4. **Durability**  
   Once a transaction is committed, the changes it made are permanent, even in the event of a system crash.

---

### Real-World Example

Imagine a bank transfer:  
Alice wants to transfer \$500 to Bob. The system must:

1. Deduct \$500 from Alice's account.  
2. Add \$500 to Bob's account.  
3. Ensure both steps succeed or neither happens (to avoid money disappearing or being created).



This is a perfect use case for transactions! 

In [ ]:
# Imports
from tabulate import tabulate
import psycopg2
import time

In [ ]:
# Database connection parameters
dbname = 'banking_db'
user = 'postgres'
password = 'postgres'
host = 'postgres_db'  # This should be the service name defined in docker-compose.yml
port = '5432'  

In [ ]:
# Establish connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

### Cursor in SQL

In SQL, a **cursor** is a database object that allows you to interact with the result set of a query in a controlled, row-by-row manner. It provides a pointer to a specific row within the result set, allowing you to process the data incrementally.

---

#### **Key Points of Cursors**:

- **Purpose**: A cursor enables you to retrieve rows from the database one by one or in chunks, and perform operations like updating, inserting, or deleting specific rows as needed.

- **How it Works**:
    - When you execute a query, the result set is generated by the database. 
    - The cursor helps navigate and interact with this result set.
    - You can use the cursor to move through the data row by row or in batches.

- **Common Operations**:
    - **Fetch Rows**: Cursors allow you to fetch one or more rows from the result set.
    - **Iterate Over Rows**: You can iterate through the result set one row at a time, making it easy to process large sets of data incrementally.
    - **Modify Data**: Cursors can be used to update, delete, or insert rows in a transaction.

---

In [ ]:
# Create a cursor object to interact with the database
cur = conn.cursor()

In [ ]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM accounts")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

## 2. SQL Transactions

### Transaction Lifecycle

1. **BEGIN TRANSACTION**:
   
   Marks the start of a transaction. Any changes made after this command are part of the transaction and are not permanent until the transaction is committed.

2. **COMMIT**

   Applies all changes made during the transaction to the database. Once committed, the changes are permanent and cannot be rolled back.

3. **ROLLBACK**
   
   Undoes all changes made during the transaction, restoring the database to its state before the transaction began. This command is used if there is an error or if the transaction needs to be discarded.

---

### SQL Transaction Commands

1. **BEGIN TRANSACTION** (or **START TRANSACTION**)
   
   Starts a new transaction. Any changes made after this command are not permanent until the transaction is committed. This command signals the beginning of a unit of work.

2. **COMMIT**
   
   Finalizes the transaction and applies all changes to the database. Once committed, the changes become permanent and are visible to other transactions.

3. **ROLLBACK**
   
   Reverses all changes made since the transaction began, undoing any changes made during the transaction. This command is typically used when an error occurs or when a transaction should not be finalized.



## 2.1. A Trivial Transaction

In [ ]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Deduct $49 from Alice's account
cur.execute("UPDATE accounts SET balance = balance - 49 WHERE name = 'Alice Smith'")

# Add $49 to Bob's account
cur.execute("UPDATE accounts SET balance = balance + 49 WHERE name = 'Bob Johnson'")

# Commit the transaction to make changes permanent
cur.execute("COMMIT;")

In [ ]:
# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

## 2.1. Rolling Back a Transaction

#### Charlie Brown's account should not go below `$7000`, as he has a pending payment of `$7000` that must be accounted for.


In [ ]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Charlie Brown'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Deduct 3001 dollars from Chris's account (this is the risky operation)
cur.execute("UPDATE accounts SET balance = balance - 3001 WHERE name = 'Charlie Brown'")

# Check Chris's new balance
cur.execute("SELECT balance FROM accounts WHERE name = 'Charlie Brown'")
chris_balance = cur.fetchone()[0]

# If Chris's balance falls below 7000 dollars (due to the deduction), rollback the transaction
if chris_balance < 7000:
    print("Error: Chris's balance cannot go below 7000 dollars. Rolling back the transaction.")
    cur.execute("ROLLBACK")  # Undo all changes made during the transaction
else:
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT")
    print("Transaction successful. Changes have been committed.")

In [ ]:
# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Charlie Brown'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

## 2.2. An Aborted Transaction

In [ ]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try:
    # Deduct 2500 dollars from Bob's account (this will raise an error if Bob doesn't have enough funds)
    cur.execute("UPDATE accounts SET balance = balance - 2500 WHERE name = 'Bob Johnson'")

    # Add 2500 dollars to Alice's account
    cur.execute("UPDATE accounts SET balance = balance + 2500 WHERE name = 'Alice Smith'")

    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    print("Transaction successful: 2500 dollars transferred from Bob Johnson to Alice Smith.")

except Exception as e:
    # If an error occurs (like insufficient funds), rollback the transaction to undo all changes
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")


In [ ]:
# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

## 2.3. System Failure during Execution

In [ ]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Deduct 1000 dollars from Bob's account (this will be rolled back if power loss occurs)
cur.execute("UPDATE accounts SET balance = balance - 1000 WHERE name = 'Bob Johnson'")

# Add 1000 dollars to Alice's account
cur.execute("UPDATE accounts SET balance = balance + 1000 WHERE name = 'Alice Smith'")

# Simulate power loss by disconnecting the database
print("Simulating power loss... Disconnecting from the database.")
conn.close()

# Wait for a few seconds (simulate time of power loss)
time.sleep(3)

In [ ]:
# Assume now, the database has been recovered from the failure and we can reconnect
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

In [ ]:
# Close the connection
cur.close()
conn.close()

## 3. Isolation Levels in SQL

### Isolation Levels in SQL Transactions

In database management systems (DBMS), **isolation levels** define the degree to which the operations in one transaction are isolated from those in other concurrent transactions. These levels determine how and when the changes made by one transaction become visible to other transactions.

The goal of isolation levels is to balance the trade-off between **performance** (in terms of concurrency) and **correctness** (in terms of avoiding anomalies like dirty reads, non-repeatable reads, and phantom reads).

#### Isolation Levels

There are four standard isolation levels defined by the SQL standard, which can be set using the `SET TRANSACTION ISOLATION LEVEL` statement. These levels control the visibility of data changes between concurrently running transactions.

1. **Read Uncommitted**
2. **Read Committed**
3. **Repeatable Read**
4. **Serializable**

---

### 1. **Read Uncommitted**
- **Description**: In this isolation level, transactions can read data that has been modified but not yet committed by other transactions. This is known as a **dirty read**.
- **Use case**: This level is rarely used in practice because it can lead to data inconsistency. However, it might be acceptable when performance is more critical than correctness.
  
  **Example**: 
  - Transaction A updates a record but has not yet committed it.
  - Transaction B reads that uncommitted data (a **dirty read**).

- **Anomalies**: 
  - **Dirty Reads**: A transaction reads uncommitted changes from another transaction.
  - **Non-repeatable Reads**: Data read by one transaction is modified by another before the first transaction completes.

---

### 2. **Read Committed**
- **Description**: In this isolation level, a transaction can only read data that has been **committed** by other transactions. This avoids dirty reads but allows non-repeatable reads.
- **Use case**: This is the default isolation level for many database systems because it strikes a good balance between data consistency and performance.
  
  **Example**: 
  - Transaction A commits an update.
  - Transaction B can read the committed data.

- **Anomalies**:
  - **Non-repeatable Reads**: A transaction reads a value, and another transaction modifies that value before the first transaction completes.

---

### 3. **Repeatable Read**
- **Description**: In this isolation level, transactions can only see data that was committed before the transaction started, and once a value is read, it will not change during the transaction's duration. This prevents **dirty reads** and **non-repeatable reads**.
- **Use case**: This level is used when data consistency is critical, and there is a need to ensure that a value read at one point in time cannot change during the transaction.

  **Example**: 
  - Transaction A reads data and locks it.
  - Transaction B cannot modify that data until Transaction A completes.

- **Anomalies**: 
  - **Phantom Reads**: New rows can be added or removed by other transactions, which would not be visible to the current transaction.

---

### 4. **Serializable**
- **Description**: This is the highest level of isolation. It ensures that transactions are executed in such a way that their effect is as if they were run **serially**, one after the other. This eliminates **dirty reads**, **non-repeatable reads**, and **phantom reads**.
- **Use case**: Use this level when data consistency is paramount and the workload can tolerate reduced performance due to strict isolation requirements.
  
  **Example**: 
  - Transaction A runs, locks the data, and no other transaction can interfere until it completes.
  - Transaction B cannot access any data that might affect Transaction A’s result until A finishes.

- **Anomalies**: 
  - **Phantom Reads** are avoided.
  - However, performance may be significantly reduced because of locking and waiting.

---

### Isolation Level Comparison

| Isolation Level     | Dirty Reads | Non-repeatable Reads | Phantom Reads | Performance Impact | Use Case                                      |
|---------------------|-------------|----------------------|---------------|--------------------|-----------------------------------------------|
| **Read Uncommitted** | Yes         | Yes                  | Yes           | High               | Low data consistency requirements             |
| **Read Committed**   | No          | Yes                  | Yes           | Medium             | Default in many DBMS, suitable for most cases |
| **Repeatable Read**  | No          | No                   | Yes           | Low                | Ensures repeatable results within the txn     |
| **Serializable**     | No          | No                   | No            | Very Low           | Highest consistency, low concurrency         |

---

### Why Choose a Specific Isolation Level?

- **Read Uncommitted**: Suitable for applications where speed is more important than accuracy. For example, reporting systems where minor inconsistencies are tolerable.
  
- **Read Committed**: Default for many systems, balancing performance and consistency. Suitable for most business applications where some degree of inconsistency is acceptable.
  
- **Repeatable Read**: Useful when you need to guarantee that data read by a transaction will not change during the transaction. This is good for applications with critical financial calculations or inventories.

- **Serializable**: Most secure, ideal for systems requiring absolute accuracy in complex transactions, but it comes with a performance penalty. It's essential when performing actions that must not interfere with each other at all, like in banking systems.

---

### Summary

- **Isolation levels** define how much one transaction’s operations are isolated from others.
- As isolation increases (from Read Uncommitted to Serializable), **data consistency** improves, but **performance** tends to degrade because of increased locking and transaction serialization.
- The **right isolation level** depends on your application’s requirements for consistency vs. concurrency.


### 3.1. Dirty Reads

**PostgreSQL** does not support the `READ_UNCOMMITTED` isolation level because it can lead to dangerous issues like **dirty reads**, where transactions can read uncommitted data that may later be rolled back. Instead, PostgreSQL uses `READ_COMMITTED`, which ensures transactions only see committed data.

On the other hand, **MySQL** does support `READ_UNCOMMITTED`, allowing dirty reads. This makes MySQL a suitable choice for demonstrating the **dirty read problem**, where one transaction can read uncommitted data from another transaction, leading to inconsistencies if the changes are rolled back.


In [ ]:
import mysql.connector

# Establish connection to MySQL
conn = mysql.connector.connect(
    host="mysql_db",
    user="user",
    password="user_password",
    database="banking_db"
)

# Create a cursor object
cur = conn.cursor()

# Execute a query to see the initial database state 
cur.execute("SELECT * FROM accounts WHERE name = 'Bob Johnson'")

# Fetch and display the result
records = cur.fetchall()
print(tabulate(records, headers=col_names, tablefmt="grid"))

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
# Create two separate database connections for two clients (conn1 and conn2)
conn1 = mysql.connector.connect(host="mysql_db", user="user", password="user_password", database="banking_db")
conn2 = mysql.connector.connect(host="mysql_db", user="user", password="user_password", database="banking_db")

# Set the transaction isolation level to READ UNCOMMITTED for conn1 and conn2
conn1.start_transaction(isolation_level='READ UNCOMMITTED')
conn2.start_transaction(isolation_level='READ UNCOMMITTED')

# Create cursors for both connections
cur1 = conn1.cursor()
cur2 = conn2.cursor()

# Start Transaction 1 (conn1): Update Bob's balance without committing
cur1.execute("UPDATE accounts SET balance = balance - 500 WHERE name = 'Bob Johnson';")
print("Transaction 1: Updated Bob's balance, but did not commit yet.")

# Start Transaction 2 (conn2): Read Bob's balance (Dirty read)
cur2.execute("SELECT name, balance FROM accounts WHERE name = 'Bob Johnson';")
bob_balance = cur2.fetchone()
print(f"Transaction 2: Bob's current balance (dirty read): {bob_balance}")

# Rollback Transaction 1 (conn1)
conn1.rollback()
print("Transaction 1: Rolled back the changes.")

# Closing connections and cursors
cur1.close()
cur2.close()
conn1.close()
conn2.close()


In [ ]:
# Establish connection to MySQL
conn = mysql.connector.connect(
    host="mysql_db",
    user="user",
    password="user_password",
    database="banking_db"
)

# Create a cursor object
cur = conn.cursor()

# Execute a query to see the initial database state 
cur.execute("SELECT * FROM accounts WHERE name = 'Bob Johnson'")

# Fetch and display the result
records = cur.fetchall()
print(tabulate(records, headers=col_names, tablefmt="grid"))

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
# Create two separate database connections for two clients (conn1 and conn2)
conn1 = mysql.connector.connect(host="mysql_db", user="user", password="user_password", database="banking_db")
conn2 = mysql.connector.connect(host="mysql_db", user="user", password="user_password", database="banking_db")

# Set the transaction isolation level to READ COMMITTED for conn1 and conn2
conn1.start_transaction(isolation_level='READ COMMITTED')
conn2.start_transaction(isolation_level='READ COMMITTED')

# Create cursors for both connections
cur1 = conn1.cursor()
cur2 = conn2.cursor()

# Start Transaction 1 (conn1): Update Bob's balance without committing
cur1.execute("UPDATE accounts SET balance = balance - 500 WHERE name = 'Bob Johnson';")
print("Transaction 1: Updated Bob's balance, but did not commit yet.")

# Start Transaction 2 (conn2): Read Bob's balance (Dirty read)
cur2.execute("SELECT name, balance FROM accounts WHERE name = 'Bob Johnson';")
bob_balance = cur2.fetchone()
print(f"Transaction 2: Bob's current balance (NON-dirty read): {bob_balance}")

# Rollback Transaction 1 (conn1)
conn1.rollback()
print("Transaction 1: Rolled back the changes.")

# Closing connections and cursors
cur1.close()
cur2.close()
conn1.close()
conn2.close()

### 3.1. Non-Repeatable Reads

In [ ]:
# Create a connection and a cursor to see the initial state of the database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

cur.close()
conn.close()

In [ ]:
from psycopg2.extensions import ISOLATION_LEVEL_READ_COMMITTED

# Create two separate database connections for two clients (conn1 and conn2)
conn1 = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
conn2 = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Set the transaction isolation level to READ COMMITTED for conn1 and conn2
conn1.set_isolation_level(ISOLATION_LEVEL_READ_COMMITTED)
conn2.set_isolation_level(ISOLATION_LEVEL_READ_COMMITTED)

# Create cursors for both connections
cur1 = conn1.cursor()
cur2 = conn2.cursor()

# Start Transaction 1 (conn1): Read Bob's balance
cur1.execute("SELECT name, balance FROM accounts WHERE name = 'Bob Johnson';")
bob_balance_initial = cur1.fetchone()
print(f"Transaction 1: Initial Bob's balance: {bob_balance_initial}")

# Start Transaction 2 (conn2): Update Bob's balance
cur2.execute("UPDATE accounts SET balance = balance + 500 WHERE name = 'Bob Johnson';")
print("Transaction 2: Updated Bob's balance.")

# Commit Transaction 2 (conn2)
cur2.execute("COMMIT;")
print("Transaction 2: Commit the changes.")

# Simulate a delay to allow Transaction 1 to read again after Transaction 2's update
time.sleep(2)

# Start Transaction 1 again: Read Bob's balance after the update
cur1.execute("SELECT name, balance FROM accounts WHERE name = 'Bob Johnson';")
bob_balance_after_update = cur1.fetchone()
print(f"Transaction 1: Bob's balance after the update (Non-Repeatable Read): {bob_balance_after_update}")

# Closing connections and cursors
cur1.close()
cur2.close()
conn1.close()
conn2.close()

In [ ]:
# Create a connection and a cursor to see the initial state of the database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

cur.close()
conn.close()

In [ ]:
from psycopg2.extensions import ISOLATION_LEVEL_REPEATABLE_READ

# Create two separate database connections for two clients (conn1 and conn2)
conn1 = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
conn2 = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Set the transaction isolation level to REPEATABLE READ for conn1 and conn2
conn1.set_isolation_level(ISOLATION_LEVEL_REPEATABLE_READ)
conn2.set_isolation_level(ISOLATION_LEVEL_REPEATABLE_READ)

# Create cursors for both connections
cur1 = conn1.cursor()
cur2 = conn2.cursor()

# Start Transaction 1 (conn1): Read Bob's balance
cur1.execute("SELECT name, balance FROM accounts WHERE name = 'Bob Johnson';")
bob_balance_initial = cur1.fetchone()
print(f"Transaction 1: Initial Bob's balance: {bob_balance_initial}")

# Start Transaction 2 (conn2): Update Bob's balance
cur2.execute("UPDATE accounts SET balance = balance + 500 WHERE name = 'Bob Johnson';")
print("Transaction 2: Updated Bob's balance.")

# Commit Transaction 2 (conn2)
cur2.execute("COMMIT;")
print("Transaction 2: Commit the changes.")

# Simulate a delay to allow Transaction 1 to read again after Transaction 2's update
time.sleep(2)

# Start Transaction 1 again: Read Bob's balance after the update
cur1.execute("SELECT name, balance FROM accounts WHERE name = 'Bob Johnson';")
bob_balance_after_update = cur1.fetchone()
print(f"Transaction 1: Bob's balance after the update (Repeatable Read): {bob_balance_after_update}")

# Closing connections and cursors
cur1.close()
cur2.close()
conn1.close()
conn2.close()

### 3.2. Phantom Reads

A **phantom read** happens when a transaction re-executes a query that returns a set of rows based on a condition and finds **new rows** (phantoms) that were inserted by another transaction during its execution.

### Isolation Levels and Phantom Reads

Isolation levels are defined in the SQL standard (like `READ COMMITTED`, `REPEATABLE READ`, and `SERIALIZABLE`), but **the standard does not strictly define how these should be implemented**.  
As a result, **each database system interprets and implements these levels differently**.

In practice, **modern databases like PostgreSQL and MySQL (InnoDB)** use techniques such as **MVCC**, **predicate locking**, and **gap locks** to prevent anomalies — including **phantom reads** — even at isolation levels where the standard says they may occur.

Because phantom reads are effectively prevented in most modern systems, **we illustrate this phenomenon conceptually rather than with executable code**.  
Legacy systems or databases with weaker isolation may still be vulnerable to phantom reads.

---

### Example Scenario

Let's say we have a table called `employees`:

| id | name    | department |
|----|---------|------------|
| 1  | Alice   | Sales      |
| 2  | Bob     | Sales      |
| 3  | Charlie | Sales      |

---

#### Transaction A:

```sql
-- Transaction A starts
SELECT * FROM employees WHERE department = 'Sales';
```

This returns **3 rows**: Alice, Bob, Charlie.

---

#### Meanwhile, Transaction B:

```sql
-- Transaction B
INSERT INTO employees (id, name, department)
VALUES (4, 'Diana', 'Sales');
-- Then commits
```

---

#### Back to Transaction A:

```sql
-- Transaction A repeats the same query
SELECT * FROM employees WHERE department = 'Sales';
```

This time, the result is **4 rows**: Alice, Bob, Charlie, and now **Diana** has appeared — like a **phantom**. 

---

### Why Does This Happen?

- Under **READ COMMITTED** isolation, each statement sees only the data that was committed before it started.
- Therefore, even within the same transaction, different executions of the same query might return different results if other transactions commit in between.

---

### How Higher Isolation Levels Prevent It

| Isolation Level     | Prevents Phantom Reads? | Notes                                                    |
|---------------------|--------------------------|----------------------------------------------------------|
| READ COMMITTED      | No                    | Each statement sees latest committed state               |
| REPEATABLE READ     | Partially             | Prevents dirty & unrepeatable reads, but phantoms may still occur |
| SERIALIZABLE        | Yes                   | Enforces full isolation, avoids phantoms via predicate locking |

---


## Questions?

If you have any questions, feel free to post them on **Moodle**

May your transactions always commit and never rollback. Happy querying! 😊